### Import CSAT Data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from pyodbc import connect
from sqlalchemy import create_engine
import urllib 
import os
from dotenv import load_dotenv

In [12]:
# Function to fetch data from our Database
def fetch_data(query):

    # Load variables from .env file, change path to your local .env file
    load_dotenv('variables.env')

    # Access variables
    server = f'{os.getenv("SERVER_NAME")}'
    database = os.getenv("DB")
    user = os.getenv("USER")
    password = os.getenv("PASSWORD")

    # Connect to Database
    conn_string = f'mssql+pyodbc://{user}:{urllib.parse.quote_plus(password)}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
    
    engine = create_engine(conn_string)
    
    df = pd.read_sql_query(query, engine)

    return df

In [13]:
query = 'SELECT * FROM FactCSATSurveyData;'

# Store qury result into a dataframe
df = fetch_data(query)

In [14]:
df.head()

,CustomerID,SurveyYear,SurveyQuarter,SurveyDate,ResponseDate,RecommendationRate,CompanyRate,UserFrequency,QualityRate,PanelUsabilityRate,ReportCapability,SurveyID
0,CRM-1516050,2022,3,2022-07-01,2022-08-26,10.0,5.0,Once a Month,5.0,5.0,I need someone from Insider team to provide me...,1
1,CRM-1516050,2022,3,2022-07-01,2022-09-27,10.0,5.0,Once a Month,5.0,5.0,I don't use it often,2
2,CRM-1470950,2022,1,2022-01-01,2022-03-25,4.0,3.0,Once a Day,2.0,3.0,I need someone from Insider team to provide me...,3
3,CRM-1470950,2022,3,2022-07-01,2022-09-30,6.0,3.0,Once a Week,4.0,4.0,I tried but could not find everything I need,4
4,CRM-1470950,2022,2,2022-04-01,2022-06-23,10.0,5.0,Once a Day,4.0,3.0,I need someone from Insider team to provide me...,5


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1989 non-null   object 
 1   SurveyYear          1989 non-null   int64  
 2   SurveyQuarter       1989 non-null   int64  
 3   SurveyDate          1989 non-null   object 
 4   ResponseDate        1989 non-null   object 
 5   RecommendationRate  1989 non-null   float64
 6   CompanyRate         1989 non-null   float64
 7   UserFrequency       1989 non-null   object 
 8   QualityRate         1989 non-null   float64
 9   PanelUsabilityRate  1989 non-null   float64
 10  ReportCapability    1989 non-null   object 
 11  SurveyID            1989 non-null   int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 186.6+ KB


### Sentiment Analysis

In [16]:
import nltk 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [18]:
# Download lexicon 
nltk.download('vader_lexicon')

# Initialize analyzer
analyzer = SentimentIntensityAnalyzer()

# Calculate sentiment scores, list as input
def sentiment_score(report):
    # Get sentiment scores for list of provide values
    sentiment = analyzer.polarity_scores(report)
    # Return just compound values 
    return sentiment['compound']

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\alvar\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [29]:
# Define scores to place them into categories
def sentiment_category(score):
    # Category list
    categories = ['Negative','Mixed Negative','Neutral','Mixed positive','Positive']

    # Since score range is (-1, 1), is transformed to (0,4) to match categories list values 
    # This is done to avoid multiple conditionals
    index = round( ( score + 1 ) * 2 )

    if score: return categories[index]

    return categories[2]

In [30]:
# Define sentiment buckets
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1'
    elif 0 <= score < 0.5:
        return '0 to 0.49'
    elif -0.5 <= score < 0:
        return '-0.5 to 0'
    else:
        return '-1 to -0.5'

In [32]:
# Calculate sentiment scores
df['SentimentScore'] = df['ReportCapability'].apply(sentiment_score)

df.head()

,CustomerID,SurveyYear,SurveyQuarter,SurveyDate,ResponseDate,RecommendationRate,CompanyRate,UserFrequency,QualityRate,PanelUsabilityRate,ReportCapability,SurveyID,SentimentScore
0,CRM-1516050,2022,3,2022-07-01,2022-08-26,10.0,5.0,Once a Month,5.0,5.0,I need someone from Insider team to provide me...,1,0.0
1,CRM-1516050,2022,3,2022-07-01,2022-09-27,10.0,5.0,Once a Month,5.0,5.0,I don't use it often,2,0.0
2,CRM-1470950,2022,1,2022-01-01,2022-03-25,4.0,3.0,Once a Day,2.0,3.0,I need someone from Insider team to provide me...,3,0.0
3,CRM-1470950,2022,3,2022-07-01,2022-09-30,6.0,3.0,Once a Week,4.0,4.0,I tried but could not find everything I need,4,0.0
4,CRM-1470950,2022,2,2022-04-01,2022-06-23,10.0,5.0,Once a Day,4.0,3.0,I need someone from Insider team to provide me...,5,0.0


In [45]:
# Assign sentiment category and bucket
df['SentimentCategory'] = df['SentimentScore'].apply(sentiment_category)

df['SentimentBucket'] = df['SentimentScore'].apply(sentiment_bucket)

df.head()

,CustomerID,SurveyYear,SurveyQuarter,SurveyDate,ResponseDate,RecommendationRate,CompanyRate,UserFrequency,QualityRate,PanelUsabilityRate,ReportCapability,SurveyID,SentimentScore,SentimentCategory,SentimentBucket
0,CRM-1516050,2022,3,2022-07-01,2022-08-26,10.0,5.0,Once a Month,5.0,5.0,I need someone from Insider team to provide me...,1,0.0,Neutral,0 to 0.49
1,CRM-1516050,2022,3,2022-07-01,2022-09-27,10.0,5.0,Once a Month,5.0,5.0,I don't use it often,2,0.0,Neutral,0 to 0.49
2,CRM-1470950,2022,1,2022-01-01,2022-03-25,4.0,3.0,Once a Day,2.0,3.0,I need someone from Insider team to provide me...,3,0.0,Neutral,0 to 0.49
3,CRM-1470950,2022,3,2022-07-01,2022-09-30,6.0,3.0,Once a Week,4.0,4.0,I tried but could not find everything I need,4,0.0,Neutral,0 to 0.49
4,CRM-1470950,2022,2,2022-04-01,2022-06-23,10.0,5.0,Once a Day,4.0,3.0,I need someone from Insider team to provide me...,5,0.0,Neutral,0 to 0.49


In [46]:
# Save to csv
df.to_csv('fact_customer_sentiment.csv', index = False)